<a href="https://colab.research.google.com/github/AarkeTripathi/Movie_recommendation/blob/master/titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
import pandas as pd
import numpy as np
import math

In [30]:
train_data=pd.read_csv('train.csv')
train_data.shape

(891, 12)

In [31]:
train_data.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [32]:
train_data.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [33]:
def normalize(x):
  mu=np.mean(x,axis=0)
  sigma=np.std(x,axis=0)
  x_norm=(x-mu)/sigma
  return x_norm

In [34]:
lst=np.array(train_data['Sex'])
for i in range(len(lst)):
  if lst[i]=='male':
    lst[i]=1
  else:
    lst[i]=0
train_data['Sex']=lst
train_data['Age']=normalize(train_data['Age'])

In [35]:
med=np.median(train_data['Age'].dropna())
train_data['Age']=train_data['Age'].fillna(med)
train_data.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [36]:
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",1,-0.530377,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,0.571831,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",0,-0.254825,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,0.365167,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",1,0.365167,0,0,373450,8.0500,NaN,S


In [37]:
x_train=train_data[['Sex','Age','Pclass']]
x_train.head()

,Sex,Age,Pclass
0,1,-0.530377,3
1,0,0.571831,1
2,0,-0.254825,3
3,0,0.365167,1
4,1,0.365167,3


In [38]:
y_train=train_data['Survived']

In [39]:
w=np.ones(len(x_train.columns))
b=1

In [40]:
def sigmoid(x):
  return 1/(1+np.exp(-x))

In [68]:
def predict(x,w,b):
  z=np.dot(w,x_train.iloc[x])+b
  return sigmoid(z)

In [63]:
def compute_cost(x, y,w,b, *argv):

    m, n = x.shape
    
    ### START CODE HERE ###
    total_cost=0
    for i in range(m):
        f_wb=predict(i,w,b)
        loss=(-y[i]*math.log(f_wb))-((1-y[i])*math.log(1-f_wb))
        total_cost=total_cost+loss
    total_cost=total_cost/m
        
    ### END CODE HERE ### 

    return total_cost

In [64]:
def compute_gradient(X, y, w, b, *argv): 
    m, n = X.shape
    dj_dw = np.zeros(w.shape)
    dj_db = 0.

    ### START CODE HERE ### 
    for i in range(m):
        f_wb = predict(i,w,b)
        
        dj_db += f_wb-y[i]
        
        for j in range(n):
            dj_dw[j] += (f_wb-y[i])*X.iloc[i][j]
            
    dj_dw = dj_dw/m
    dj_db = dj_db/m
    ### END CODE HERE ###
    
    return dj_db, dj_dw

In [65]:
def gradient_descent(X, y, w_in, b_in, cost_function, gradient_function, alpha, num_iters, lambda_): 
    
    # number of training examples
    m = X.shape[0]
    
    # An array to store cost J and w's at each iteration primarily for graphing later
    J_history = []
    w_history = []
    
    for i in range(num_iters):

        # Calculate the gradient and update the parameters
        dj_db, dj_dw = gradient_function(X, y, w_in, b_in, lambda_)   

        # Update Parameters using w, b, alpha and gradient
        w_in = w_in - alpha * dj_dw               
        b_in = b_in - alpha * dj_db              
       
        # Save cost J at each iteration
        
    return w_in, b_in #return w and J,w history for graphing

In [70]:
iterations = 500
alpha = 0.01

w,b= gradient_descent(x_train ,y_train, w, b,compute_cost, compute_gradient, alpha, iterations, 0)

In [71]:
compute_cost(x_train,y_train,w,b)

0.5978723261223347

In [72]:
def prediction(x,w,b):
  arr=[]
  for i in range(x.shape[0]):
    f=predict(i,w,b)
    if f>0.5:
      arr.append(1)
    else:
      arr.append(0)
  return arr

In [73]:
predicted_lst=prediction(x_train,w,b)

In [74]:
test_data=pd.read_csv('test.csv')

In [75]:
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [77]:
lst1=np.array(test_data['Sex'])
for i in range(len(lst1)):
  if lst1[i]=='male':
    lst1[i]=1
  else:
    lst1[i]=0
test_data['Sex']=lst1
test_data['Age']=normalize(test_data['Age'])

In [78]:
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",1,0.298549,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",0,1.181328,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",1,2.240662,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",1,-0.231118,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",0,-0.584229,1,1,3101298,12.2875,NaN,S


In [80]:
x_test=test_data[['Sex','Age','Pclass']]

In [82]:
pred_lst=prediction(x_test,w,b)

In [83]:
pred_lst[:5]

[0, 1, 0, 1, 0]

In [84]:
compar=pd.read_csv('gender_submission.csv')

In [85]:
comp_lst=np.array(compar['Survived'])

In [86]:
cout=0
for i in range(x_test.shape[0]):
  if pred_lst[i]==comp_lst[i]:
    cout+=1
print(cout)

246


In [87]:
cout/x_test.shape[0]

0.5885167464114832